# Motion correction + Online CNMF/CNMF_E code

Complete pipeline for online processing using CaImAn Online, but with motion correction and CNMF/CNMF_E separate instead of done simultaneously.

This allows to visualize if motion correction was well performed prior to start CNMF_E. If parameters are set and recordings are stable, one can use just OnACID, bu may not be advisable.

## Initialization

### Import packages

In [1]:
import logging
try:
    if __IPYTHON__:
        # this is used for debugging purposes only. allows to reload classes when changed
        get_ipython().magic('load_ext autoreload')
        get_ipython().magic('autoreload 2')
except NameError:
    pass

logging.basicConfig(format=
                          "%(relativeCreated)12d [%(filename)s:%(funcName)20s():%(lineno)s] [%(process)d] %(message)s",
                    # filename="/tmp/caiman.log",
                    level=logging.INFO)

import numpy as np
import os

import caiman as cm
import mesmerize_core as mescore
from caiman.source_extraction import cnmf
import matplotlib.pyplot as plt
from fastplotlib import ImageWidget, Plot, GridPlot
from fastplotlib.graphics.line_slider import LineSlider
from ipywidgets import VBox, IntSlider, Layout

import cv2

C:\Users\Gebruiker\AppData\Local\Temp\ipykernel_8344\1931500065.py:5: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  get_ipython().magic('load_ext autoreload')
C:\Users\Gebruiker\AppData\Local\Temp\ipykernel_8344\1931500065.py:6: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  get_ipython().magic('autoreload 2')


### Set data

Data can be set in a loop but should be set as a list and not as a string. 
Otherwise the code will turn them into a list.

Using '/' instead of '\\' allows to pass this code among operative systems. Linux and 

In [2]:
fnames = ['Z:/Shared/For_Rui/original images of one session/MC Results/12937-06AI2_3DPI_S_MC.tif']

if isinstance(fnames,str):
    fnames = [fnames]
    
for i in range(0,len(fnames)):
    if os.path.exists(fnames[i]) == False:
        logging.error('File '+ str(i+1) + ': '+fnames[i] + ' does not exits! Please correct the filename.')

## Motion correction

### Set motion correction parameters.

All the parameters are commented. However, the most important parameters in this code are:

<ul>
    <li><b> gSig_filt: </b> Gaussian filter. Should normally be half the neuron size.</li>
    <li><b> pw_rigid: </b> Set True for piece-wise motion correction.</li>
    <li><b> max_shifts_online: </b> Maximum rigid shifts. It should not be huge to not have major problems in the updated templates.</li>
</ul>

In [ ]:
##############################################################################################################
#Set parameters
gSig_filt = (9,9)                 # Gaussian filter size. It should normally be half the neuron size.
pw_rigid = False                    # True if piece-wise motion correction should be used
strides = (300, 180)                  # Patch sizes in pixels.
overlaps = (24, 24)                 # Overlap between patches in pixels. This value normally does not have to be very high.
max_shifts_online = 50              # Maximum rigid shifts
max_deviation_rigid = 10            # Maximum deviation in pixels between rigid shifts and shifts of individual patches
border_nan = 'copy'                 # Flag for allowing NaN in the boundaries. True allows NaN, whereas 'copy' copies the value of the nearest data point.
shifts_opencv = True                # Flag for applying shifts using cubic interpolation (otherwise FFT)
init_batch = 500                    # Initial batch 
ds_factor = 1                       # Spatial binning factor. Have in account that if spatial binning is used, all values above are unaltered.
normalize = False                   # Whether to normalize each frame prior to online processing. Set it to False, otherwise results may be wierd sometimes

params_dict = {'fnames': fnames,
               'gSig_filt': gSig_filt,
               'pw_rigid': pw_rigid,
               'strides': strides,
               'overlaps': overlaps,
               'max_shifts_online': max_shifts_online,
               'max_deviation_rigid': max_deviation_rigid,
               'border_nan': border_nan,
               'shifts_opencv': shifts_opencv,
               'init_batch': init_batch,
               'ds_factor': ds_factor,
               'normalize': normalize}

#Set parameters
opts = cnmf.params.CNMFParams(params_dict=params_dict)
cnm = cnmf.online_cnmf.OnACID(params=opts)  

nbits = np.uint8                   # Number of bits in which to save final images as tiff files.
init_template = None                 # If an initial template can be provided, this should be where to set this variable into the template 2D numpy array.

### Run motion correction online in each file.

This code uses the same template for 2nd and subsequent files if <b> reuse_template </b> is set to True

In [ ]:
reuse_template = False
MCfnames = fnames.copy()

for i in range(0,len(fnames)):
    cnm.params.set('data',{'fnames':[fnames[i]]})
    cnm.motion_correction_online(template = init_template, save_movie=True,nbits=nbits)

    MCfnames[i] = os.path.join(os.path.dirname(fnames[i]),'MC Results', os.path.splitext(os.path.basename(fnames[i]))[0]+str('_MC.tif'))
    
    if reuse_template:
        init_template = cm.load(os.path.join(os.path.dirname(fnames[i]), 'MC_results', 'MC_templates.tif'))[-1]

# Check motion corrected file

If you had a list of several files, select the index of files to compare (the first file has index=0).

In [ ]:
i = 0                         #Index of files to compare

originalMovie = mescore.movie_readers.default_reader(fnames[i])
MC_movie = mescore.movie_readers.default_reader(MCfnames[i])

mcorr_iw = ImageWidget(
    data=[originalMovie, MC_movie], 
    vmin_vmax_sliders=True, 
    cmap="gray"
)

mcorr_iw.show()

## Close Canvas to not make computer slow

In [ ]:
mcorr_iw.plot.canvas.close()

# Set up parameters for CNMF and CNMF_E

<b> Below you have parameters optimized for 1-photon and 2-photon separatelly. Choose the only that best suits you </>
    
Online calcium analysis can be done in patches. However that is normally not necessary.
In this case, patches are not being used.
Moreover, the preprocessing of images is removed in this example.

For a good online analysis, it is important to define a good initialization. That depends on some variables such as:

<ul>
    <li><b>init_batch: </b> It says the number of frames used for initializing the spatial components. Decrease if the kernel dies due to lack of memory.</li>
    <li><b>K: </b> Number of components to find in initialization. If set to None, the code tries to find based on a matemathical calculus as if running the normal CNMF pipeline.</li>
    <li><b>gSig: </b> Average radius of neurons.</li>
    <li><b>gSiz: </b> Half size of neuron bounding box. This value does not need to be initialized unless when one wants to detect non round shapes, such as dendrites.</li>
    <li><b>center_psf: Set true for 1 photon data. Set false for 2 photon data </b></li>
    <li><b>method_init: </b> Method for detect initialspatial components. <b>Should be set as 'corr_pnr' for 1 photon and 'greedy_roi' for 2 photon. </b></li>
    <li><b>init_method: </b> Method for initialize components. <b>It can be 'bare' suitable for 1 photon, 'seeded' when neurons are already seeded or 'cnmf' suitable for 2 photon </b>. </li>
</ul>

## These first parameters were optimized in 2-photon recordings

Change the parameters <b> s_min </b> if necessary to change the threshold.

In [3]:
#INITIALIZATION VARIABLES

#Choose which file to run CNMF on
analyzeFname = [MCfnames[0]]
if isinstance(analyzeFname,str):
    analyzeFname = [analyzeFname]

####################################################################################################################################################
init_batch = 3000                                                   # number of frames to initialize components
K = 50                                                               # initial number of components
center_psf = False                                                   # wether to use 1 photon (True) or 2 photon (False) analysis
ssub = 1                                                            # satial subsampling
ssub_B = 1                                                          # downsampling factor for background during corr_pnr
tsub = 1                                                            # temporal subsampling
nb = 2                                                              # number of background components. Set to 2 if background fluctuates a lot.
method_init = 'greedy_roi'                                          # Method for initialize spatial components. Use 'greedy_roi' for 2-photon and 'corr_pnr'for 1 photon
init_method = 'cnmf'                                                # initialization method. It can be 'bare', 'cnmf' or 'seeded'
ring_size_factor = 1.5                                              # Radius of ring (*gSig) for computing backgroud corr_pnr

#SPATIAL COMPONENTS VARIABLES
####################################################################################################################################################
gSig = (3,3)                                                        # expected half size of neurons
gSiz = (9,9)                                                       # half-size of bounding box for each neuron. This value is by default 2*gSig + 1
min_corr = 0.8                                                      # minimum correlation value for accepting spatial footprint in corr_pnr method
min_pnr = 0.7                                                       # minimum peak to noise ratio value for accepting spatial footprint in corr_pnr method
method_exp = 'dilate'                                               # method for expanding footprint of spatial components: 'ellipse' or 'dilate'
dist = 3                                                            # expansion factor of ellipse

#TEMPORAL COMPONENTS VARIABLES
####################################################################################################################################################
fr = 60                                                             # frame rate (Hz)
decay_time = 0.4                                                    # approximate length of transient event in seconds
p = 0                                                               # order of AR indicator dynamics
s_min = -5                                                          # Minimum spike threshold amplitude (computed in the code if is set to None but may lead to errors).
optimize_g = True

#ONLINE PROCESS VARIABLES
####################################################################################################################################################
expected_comps = 200                                                # maximum number of expected components used for memory pre-allocation (exaggerate here)
min_SNR = 1                                                         # minimum SNR for accepting new components
rval_thr = 0.90                                                     # correlation threshold for new component inclusion
#sniper_mode = True                                                 # flag using a CNN to detect new neurons (o/w space correlation is used)
dist_shape_update = True                                            # flag for updating shapes in a distributed way
min_num_trial = 10                                                  # number of candidate components per frame     
epochs = 2                                                          # number of passes over the data
show_movie = False                                                  # show the movie with the results as the data gets processed
stop_detection = True                                              # stop detection on the last epoch

params_dict = {'fnames': analyzeFname,
               'init_batch': init_batch,
               'K': K,
               'center_psf': center_psf,
               'ssub': ssub,
               'ssub_B': ssub_B,
               'tsub': tsub,
               'nb': nb,
               'method_init': method_init,
               'init_method': init_method,
               'ring_size_factor': ring_size_factor,
               
               'gSig': gSig,
               'gSiz': gSiz,
               'min_corr': min_corr,
               'min_pnr': min_pnr,
               'method_exp': method_exp,
               'dist': dist,
               
               'fr': fr,
               'decay_time': decay_time,
               'p': p,
               's_min': s_min,
               'optimize_g': optimize_g,
               
               'expected_comps': expected_comps,
               'min_SNR': min_SNR,
               'rval_thr': rval_thr,
               #'sniper_mode': sniper_mode,
               'dist_shape_update': dist_shape_update,
               'min_num_trial': min_num_trial,
               'epochs': epochs,
               'show_movie': show_movie,
               'stop_detection': stop_detection,

               'motion_correct': False,
               'normalize': True,
               'n_pixels_per_process':200}
opts = cnmf.params.CNMFParams(params_dict=params_dict)

       38785 [params.py:                 set():976] [8344] Changing key fnames in group data from None to ['Z:/Shared/For_Rui/original images of one session/MC Results/12937-06AI2_3DPI_S_MC.tif']
       38787 [params.py:                 set():976] [8344] Changing key fr in group data from 30 to 60
       38789 [params.py:                 set():976] [8344] Changing key p in group preprocess from 2 to 0
       38789 [params.py:                 set():976] [8344] Changing key n_pixels_per_process in group preprocess from None to 200
       38791 [params.py:                 set():976] [8344] Changing key K in group init from 30 to 50
       38791 [params.py:                 set():976] [8344] Changing key ssub in group init from 2 to 1
       38792 [params.py:                 set():976] [8344] Changing key ssub_B in group init from 2 to 1
       38794 [params.py:                 set():976] [8344] Changing key tsub in group init from 2 to 1
       38794 [params.py:                 set():976] 

## These second parameters were optimized for 1-photon images

### First: Plot correlation and peak to noise ratio images to help chosing the values

Set a gSig value, initial batch of images, spatial downsampling and temporal downsampling.
All these parameters can then be set again for running CNMF_E, with the optimized ones.

In [ ]:
#Choose which file to run CNMF_E on
analyzeFname = [MCfnames[0]]
if isinstance(analyzeFname,str):
    analyzeFname = [analyzeFname]
    
#
gSig = (6, 6)          # expected half size of neurons 
init_batch = 3000      # number of frames to initialize components 
ssub = 2               # spatial subsampling
tsub = 2               # temporal subsampling


images = cm.load(analyzeFname, subindices = slice(0,init_batch,tsub))
if ssub>1:
    images = images.resize(1./ssub, 1./ssub)

cn_filter, pnr = cm.summary_images.correlation_pnr(images, gSig=gSig[0], swap_dim=False) # change swap dim if output looks weird, it is a problem with tiffile
# inspect the summary images and set the parameters

cm.utils.visualization.inspect_correlation_pnr(cn_filter, pnr)


### Now choose parameters for CNMF_E

<b> Don't forget to change here the parameters optimized above. </b>
<p> Change the parameters <b> s_min </b> if necessary to change the threshold.</p>
<p> Change the parameter <b> ring_size_factor <b/> if necessary to change the background correction. </p>

In [22]:
#INITIALIZATION VARIABLES

analyzeFname = [fnames[0]]
if isinstance(analyzeFname,str):
    analyzeFname = [analyzeFname]
    
####################################################################################################################################################
init_batch = 3000                                                   # number of frames to initialize components
K = None                                                            # initial number of components
center_psf = True                                                   # wether to use 1 photon (True) or 2 photon (False) analysis
ssub = 2                                                            # spatial subsampling
ssub_B = 2                                                          # downsampling factor for background during corr_pnr
tsub = 2                                                            # temporal subsampling
nb = 0                                                              # number of background components. Set to 2 if background fluctuates a lot.
method_init = 'corr_pnr'                                            # Method for initialize spatial components. Use 'greedy_roi' for 2-photon and 'corr_pnr'for 1 photon
init_method = 'cnmf'                                                # initialization method. It can be 'bare', 'cnmf' or 'seeded'
ring_size_factor = 1.5                                                # Radius of ring (*gSig) for computing backgroud corr_pnr

#SPATIAL COMPONENTS VARIABLES
####################################################################################################################################################
gSig = (9,9)                                                        # expected half size of neurons
gSiz = (23,23)                                                       # half-size of bounding box for each neuron. This value is by default 2*gSig + 1
min_corr = 0.85                                                      # minimum correlation value for accepting spatial footprint in corr_pnr method
min_pnr = 20                                                        # minimum peak to noise ratio value for accepting spatial footprint in corr_pnr method
method_exp = 'dilate'                                               # method for expanding footprint of spatial components: 'ellipse' or 'dilate'
dist = 3                                                            # expansion factor of ellipse

#TEMPORAL COMPONENTS VARIABLES
####################################################################################################################################################
fr = 30                                                             # frame rate (Hz)
decay_time = 0.1                                                    # approximate length of transient event in seconds
p = 2                                                               # order of AR indicator dynamics
s_min = -12                                                          # Minimum spike threshold amplitude (computed in the code if is set to None but may lead to errors).
optimize_g = True

#ONLINE PROCESS VARIABLES
####################################################################################################################################################
expected_comps = 200                                                # maximum number of expected components used for memory pre-allocation (exaggerate here)
min_SNR = 3                                                         # minimum SNR for accepting new components
rval_thr = 0.90                                                     # correlation threshold for new component inclusion
#sniper_mode = True                                                 # flag using a CNN to detect new neurons (o/w space correlation is used)
dist_shape_update = True                                            # flag for updating shapes in a distributed way
min_num_trial = 10                                                  # number of candidate components per frame     
epochs = 1                                                          # number of passes over the data
show_movie = False                                                  # show the movie with the results as the data gets processed
stop_detection = True                                               # stop detection on the last epoch
update_freq = 200                                                 # frequency of frames to update spatial shapes

params_dict = {'fnames': analyzeFname,
               'init_batch': init_batch,
               'K': K,
               'center_psf': center_psf,
               'ssub': ssub,
               'ssub_B': ssub_B,
               'tsub': tsub,
               'nb': nb,
               'method_init': method_init,
               'init_method': init_method,
               'ring_size_factor': ring_size_factor,
               
               'gSig': gSig,
               'gSiz': gSiz,
               'min_corr': min_corr,
               'min_pnr': min_pnr,
               'method_exp': method_exp,
               'dist': dist,
               
               'fr': fr,
               'decay_time': decay_time,
               'p': p,
               's_min': s_min,
               'optimize_g': optimize_g,
               
               'expected_comps': expected_comps,
               'min_SNR': min_SNR,
               'rval_thr': rval_thr,
               #'sniper_mode': sniper_mode,
               'dist_shape_update': dist_shape_update,
               'min_num_trial': min_num_trial,
               'epochs': epochs,
               'show_movie': show_movie,
               'stop_detection': stop_detection,
               'update_freq': update_freq,

               'motion_correct': False,
               'normalize': False,
               'n_pixels_per_process':200}
opts = cnmf.params.CNMFParams(params_dict=params_dict)

    47257115 [params.py:                 set():976] [8344] Changing key fnames in group data from None to ['Z:/Shared/For_Rui/original images of one session/MC Results/12937-06AI2_3DPI_S_MC.tif']
    47257116 [params.py:                 set():976] [8344] Changing key decay_time in group data from 0.4 to 0.1
    47257118 [params.py:                 set():976] [8344] Changing key n_pixels_per_process in group preprocess from None to 200
    47257118 [params.py:                 set():976] [8344] Changing key K in group init from 30 to None
    47257118 [params.py:                 set():976] [8344] Changing key center_psf in group init from False to True
    47257119 [params.py:                 set():976] [8344] Changing key nb in group init from 1 to 0
    47257119 [params.py:                 set():976] [8344] Changing key method_init in group init from greedy_roi to corr_pnr
    47257122 [params.py:                 set():976] [8344] Changing key gSig in group init from [5, 5] to (9, 9)
 

# Initialize spatial components in a subset of the frames

Initialize parameters and run the initialization to create and refine spatial components.
This code does not use the ring model CNN.

In [23]:
cnm = cnmf.online_cnmf.OnACID(params=opts)
cnm.initialize_online()

cnm.save(os.path.join(os.path.dirname(analyzeFname[0]), "Initialized neurons.hdf5"))

    47259810 [movies.py:                load():1605] [8344] Your tif file is multiseries. Performance may be affected.
    47295512 [online_cnmf.py:   initialize_online():1222] [8344] Frame size:(480, 752)
    47312445 [params.py:                 set():976] [8344] Changing key n_processes in group patch from 1 to 15
    47312979 [cnmf.py:                 fit():450] [8344] Parallel processing in a single patch is not available for loaded in memory or sliced data.
    47329546 [cnmf.py:                 fit():463] [8344] (3000, 480, 752)
    47329546 [cnmf.py:                 fit():480] [8344] Using 15 processes
    47329547 [cnmf.py:                 fit():491] [8344] using 200 pixels per process
    47329548 [cnmf.py:                 fit():492] [8344] using 5000 block_size_spat
    47329549 [cnmf.py:                 fit():493] [8344] using 5000 block_size_temp
    47329549 [cnmf.py:                 fit():496] [8344] preprocessing ...
    47329550 [pre_processing.py:interpolate_missing_da

# Verify initial components shapes and traces

Check the spatial footprints on top of the spatial correlation, peak to noise and mean images. When clicking on spatial footprints check traces.

First create function to click on spatial footprints and check traces.

#### If necessary open previously saved shapes

In [6]:
cnmOpened = cnmf.online_cnmf.load_OnlineCNMF(os.path.join(os.path.dirname(fnames[0]), "Initialized neurons.hdf5"))
cnmOpened.params.set('data',{'fnames':cnmOpened.params.get('data','fnames').astype('U')})
cnmOpened.params.set('temporal',{'solvers':cnmOpened.params.get('temporal','solvers').astype('U')})

       82183 [params.py:                 set():976] [8344] Changing key decay_time in group data from 0.4 to 0.1
       82184 [params.py:                 set():976] [8344] Changing key dims in group data from None to (480, 752)
       82185 [params.py:                 set():976] [8344] Changing key fnames in group data from None to [b'Z:/Shared/For_Rui/original images of one session/MC Results/12937-06AI2_3DPI_S_MC.tif']
       82185 [params.py:                 set():976] [8344] Changing key method_init in group init from greedy_roi to corr_pnr
       82186 [params.py:                 set():976] [8344] Changing key K in group init from 30 to None
       82186 [params.py:                 set():976] [8344] Changing key center_psf in group init from False to True
       82187 [params.py:                 set():976] [8344] Changing key gSig in group init from [5, 5] to [6 6]
       82189 [params.py:                 set():976] [8344] Changing key gSiz in group init from (11, 11) to [23 23]
 

In [7]:
# Function to click on spatial footprints
def euclidean(source, target, event, new_data):
    """maps click events to contour"""
    # calculate coms of line collection
    indices = np.array(event.pick_info["index"])
    
    coms = list()

    for contour in target.graphics:
        coors = contour.data()[~np.isnan(contour.data()).any(axis=1)]
        com = coors.mean(axis=0)
        coms.append(com)

    # euclidean distance to find closest index of com 
    indices = np.append(indices, [0])
    
    ix = int(np.linalg.norm((coms - indices), axis=1).argsort()[0])
    
    target._set_feature(feature="colors", new_data=new_data, indices=ix)
    
    return None

### Compute analysis of good and bad components

Use caiman function of component evaluation to check good and bad components

In [46]:
cnm.estimates.evaluate_components(cm.load(cnm.params.get('data','fnames')[0:init_batch]).transpose(1,2,0), cnm.params)

  0%|                                                                                            | 0/1 [07:45<?, ?it/s]


MemoryError: Unable to allocate 60.5 GiB for an array with shape (45000, 480, 752) and data type float32

### Show spatial footprints and traces

Now load initial batch, compute spatial footprints and temporal footprints of each cell and background and check if traces match.

In [44]:
# Get the analyzed movie
movie = mescore.movie_readers.default_reader(analyzeFname[0])[0:init_batch]
reconstructed_movie = mescore.caiman_extensions.cnmf.LazyArrayRCM(cnm.estimates.A,cnm.estimates.C, cnm.estimates.dims)

# Get the spatial components and temporal components
contours = cm.utils.visualization.get_contours(cnm.estimates.A,cnm.params.get('data','dims'))
coordinates = list()
centers = list()
for contour in contours:
        coors = contour["coordinates"]
        coors = coors[~np.isnan(coors).any(axis=1)]
        coordinates.append(coors)

        center = coors.mean(axis=0)
        centers.append(center)

temporal = cnm.estimates.C                                     # Temporal components
background = cnm.estimates.YrA                                 # Background components

In [37]:
# for the image data and contours
#iw_initialization = ImageWidget(reconstructed_movie, vmin_vmax_sliders=False, cmap="gray")

initialization_grid = GridPlot(
    #shape=(1, 3), 
    #controllers=[[0, 0, 0]], 
    #names=[["Movie", "Reconstructed movie", "Residuals"]]
    shape = (1,2),
    controllers=[[0, 0]],
    names=[["Reconstructed movie", "Residuals"]]
)

b0 = np.array(cnm.estimates.b0.reshape(cnm.estimates.dims, order='F'))

#movie_graphic = initialization_grid["Movie"].add_image(movie[0], cmap="gray")
reconstructed_graph = initialization_grid["Reconstructed movie"].add_image(reconstructed_movie[0], cmap="gnuplot2")
residual_graph = initialization_grid["Residuals"].add_image(movie[0]-reconstructed_movie[0]-b0, cmap="gnuplot2")

# add good contours to the plot within the widget
contours_graphic = initialization_grid["Reconstructed movie"].add_line_collection(coordinates, colors="red", name="contours")
#contours_graphic = initialization_grid["Reconstructed movie"].add_line_collection(coordinates, colors="cyan", name="contours")

# temporal plot
plot_temporal = Plot()
background_graphic = plot_temporal.add_line_collection(background+temporal, colors="red", name="background")
temporal_graphic = plot_temporal.add_line_collection(temporal, colors="gray", name="temporal")
#SpikeTrace = initialization_grid["Spikes"].add_line_collection(temporal, colors="cyan", name="temporal")

# a vertical line that is syncronized to the image widget "t" (timepoint) slider
#_ls = LineSlider(x_pos=0, bounds=(temporal.min(), temporal.max()), slider=iw_initialization.sliders["t"])
#plot_temporal.add_graphic(_ls)


# since this is a GridPlot and not an ImageWidget we need to define sliders
slider = IntSlider(min=0, max=movie.shape[0] - 1, value=0, step=1)

# vertical line sliders
_ls = LineSlider(x_pos=0, bounds=(temporal.min(), temporal.max()), slider=slider)
#_ls2 = LineSlider(x_pos=0, bounds=(temporal.min(), temporal.max() + temporal_stack.items[-1].position.y), slider=slider)
plot_temporal.add_graphic(_ls)
#initialization_grid["Spikes"].add_graphic(_ls)

plot_static = Plot()
static_background_graphic = plot_static.add_line_collection(background, colors="yellow", name="background")
static_temporal_graphic = plot_static.add_line_collection(temporal, colors="cyan", name="temporal")

# function to update each frame
def update_frame(change):
    ix = change["new"]
    #movie_graphic.data = movie[ix]
    reconstructed_graph.data = reconstructed_movie[ix]
    residual_graph.data = movie[ix]-reconstructed_movie[ix]-b0
    
slider.observe(update_frame, "value")

@plot_temporal.renderer.add_event_handler("resize")
def update_slider_width(*args):
    width, h = plot_temporal.renderer.logical_size
    slider.layout = Layout(width=f"{width}px")
    
    
# stack them
#VBox([plot_static.show(),plot_temporal.show(), initialization_grid.show(), slider])
VBox([plot_temporal.show(), initialization_grid.show(), slider])

RFBOutputContext()

RFBOutputContext()

RFBOutputContext()

In [ ]:
#Auto scale
plot_temporal.auto_scale()
plot_temporal.camera.scale.x = 0.85

In [18]:
# To view one by one, first hide all traces
temporal_graphic[:].present = False
background_graphic[:].present = False

#image_graphic = initialization_grid['Movie'].plot["image"]
#image_graphic = initialization_grid['Movie']

# link image to contours
reconstructed_graph.link(
    "click",
    target=contours_graphic,
    feature="colors", 
    new_data="w", 
    callback=euclidean
)

# link contour color changes (which are triggered by the click events as defined above) to everything else

# thickness of contour
contours_graphic.link("colors", target=contours_graphic, feature="thickness", new_data=5)

# toggle temporal component when contour changes color
contours_graphic.link("colors", target=temporal_graphic, feature="present", new_data=True)
contours_graphic.link("colors", target=background_graphic, feature="present", new_data=True)
# autoscale temporal plot to the current temporal component
temporal_graphic[:].present.add_event_handler(plot_temporal.auto_scale)
background_graphic[:].present.add_event_handler(plot_temporal.auto_scale)

C:\Users\Gebruiker\mambaforge\envs\caimanRui\lib\site-packages\fastplotlib\graphics\features\_base.py:87: UserWarning: Event handler <bound method Interaction.event_handler of 'contours': LineCollection @ 0x1817fe09960
Collection of <51> Graphics> is already registered.
  warn(f"Event handler {handler} is already registered.")


## Close Canvas to not make computer slow

In [29]:
plot_temporal.canvas.close()
plot_static.canvas.close()
initialization_grid.canvas.close()
slider.close()

# When the initialization is as expected, run remaining frames

In [ ]:
cnm.deconvolve_pre_initialized()

# Run Online calcium analysis

In [ ]:
cnm = cnmf.online_cnmf.OnACID(params=opts)
cnm.fit_online()

cnm.save(os.path.join(os.path.dirname(analyzeFname[0]), "Final neurons.hdf5"))

# Optionally save results and do some plotting

In [26]:
logging.info('Number of components: ' + str(cnm.estimates.A.shape[-1]))
#Cn = cm.load(fnames[0], subindices=slice(0,500)).local_correlations(swap_dim=False)
#cnm.estimates.plot_contours(img=Cn)
cnm.estimates.nb_view_components(img=None, denoised_color='red')

    48152620 [2247812530.py:            <module>():1] [8344] Number of components: 168


In [ ]:
# we can get the contours of the spatial components
contours, coms = cnm.get_contours("all", swap_dim=False)

# and temporal components
temporal = cnm.get_temporal("all")

ixs_good = cnm.get_good_components()
ixs_bad = cnm.get_bad_components()